In [42]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D


In [43]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv('data/test.csv')
data_test_survive = pd.read_csv('data/gender_submission.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [44]:

train_data=train_data.replace(['female','male'],[0,1])
train_data=train_data.replace(['S','C','Q'],[0,1,2])
tarin_data=train_data.fillna(0,inplace=True)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,0.0


In [45]:
Y_train=train_data[['Survived']]
X_train=train_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
Y_train=np.array(Y_train)
X_train=np.array(X_train)
print(X_train.shape,Y_train.shape)

(891, 7) (891, 1)


In [58]:
test_data = test_data.replace(['female', 'male'], [0, 1])
test_data = test_data.replace(['S', 'C', 'Q'], [0, 1, 2])
test_data = test_data.fillna(0)
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,0,2
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,0,0
2,894,2,"Myles, Mr. Thomas Francis",1,62.0,0,0,240276,9.6875,0,2
3,895,3,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.0,1,1,3101298,12.2875,0,0


In [59]:
X_test = test_data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
Y_test = data_test_survive[['Survived']]
print(X_test.shape,Y_test.shape)


(418, 7) (418, 1)


In [47]:

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(10).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(8)

In [48]:
class Model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.dense1 = Dense(8,activation="relu")
    self.dense2 = Dense(18,activation="relu")
    self.dense3 = Dense(2 ,activation = "softmax")

  def call(self,x):
     x = self.dense1(x)
     x = self.dense2(x)
     x = self.dense3(x)
     return x

model = Model()

In [49]:
loss_functio = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [50]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [51]:
@tf.function
def train(X_train,Y_train):
    with tf.GradientTape() as tape:
        pred = model(X_train)
        loss = loss_functio(Y_train,pred)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(Y_train,pred)

In [52]:
@tf.function
def test(X_test,Y_test):
        pred = model(X_test)
        loss = loss_functio(Y_test,pred)
        test_loss(loss)
        test_accuracy(Y_test,pred)

In [57]:
epochs = 30
for epoch in range(epochs):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for X_train, Y_train in train_dataset:
    train(X_train, Y_train)

  for X_test,Y_test in test_dataset:
    test(X_test, Y_test)

  print("epoch :" ,epoch + 1,
        f"Train Loss :{train_loss.result()}",
        f"Train accuracy:{train_accuracy.result()}",
        f"Test Loss :   {test_loss.result()}",
        f"Test accuracy:{test_accuracy.result()}")


epoch : 1 Train Loss :0.41189494729042053 Train accuracy:0.8237934708595276 Test Loss :   0.30466943979263306 Test accuracy:0.9043062329292297
epoch : 2 Train Loss :0.4130264222621918 Train accuracy:0.8271604776382446 Test Loss :   0.3008025288581848 Test accuracy:0.9043062329292297
epoch : 3 Train Loss :0.41302260756492615 Train accuracy:0.8249158263206482 Test Loss :   0.303039014339447 Test accuracy:0.9043062329292297
epoch : 4 Train Loss :0.4102991223335266 Train accuracy:0.8282828330993652 Test Loss :   0.3055799901485443 Test accuracy:0.9019138813018799
epoch : 5 Train Loss :0.4112683832645416 Train accuracy:0.8282828330993652 Test Loss :   0.3102373480796814 Test accuracy:0.8971291780471802
epoch : 6 Train Loss :0.41097283363342285 Train accuracy:0.8305274844169617 Test Loss :   0.30440136790275574 Test accuracy:0.9043062329292297
epoch : 7 Train Loss :0.41168370842933655 Train accuracy:0.8282828330993652 Test Loss :   0.30160486698150635 Test accuracy:0.89952152967453
epoch : 8

In [60]:
model.save("model_Titanic")
model = tf.keras.models.load_model("model_Titanic")

INFO:tensorflow:Assets written to: model_Titanic\assets


INFO:tensorflow:Assets written to: model_Titanic\assets
